In [ ]:
from google.colab import output
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!gdown 1TC5lvMLoXMc4waX-BQCoUFe8l8JMdcev
!gdown 16OsGtfwFgV6MVfKXadxRqU0KEOe0E93k
!gdown 1is-2mxPrh_ipjbmWUwxaepFFKGJC1HSo
!unzip boards2.zip
!unzip boards1.zip
!pip install ultralytics
output.clear()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import skimage
import yaml
import tqdm
import os
from PIL import Image
import collections
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from ultralytics import YOLO
import torch
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms, utils
from torchvision.models import resnet18, ResNet18_Weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped


def load_board_extractor_state(path):
    with open(path) as f:
        board_annotation = yaml.load(f, Loader=yaml.FullLoader)

    return board_annotation

## Board recognition

In [ ]:
ROOT = 'dataset'

def read_annot(path):
    with open(path) as f:
        annot = f.read()
    return annot

def get_dataset(path):
    dataset = []
    for dirname, _, filenames in tqdm.tqdm(os.walk(path)):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.JPG') or filename.endswith('.png'):
                row = {}
                image = cv2.imread(os.path.join(dirname, filename))
                row['image'] = image
                row['path'] = os.path.join(dirname, filename)
                if os.path.exists(os.path.join(dirname, 'board_extractor_state.yml')):
                    row['pts_clicks'] = load_board_extractor_state(os.path.join(dirname, 'board_extractor_state.yml'))['pts_clicks']
                dataset.append(row)
    return dataset
dataset = get_dataset(ROOT)

41it [01:14,  1.82s/it]


In [ ]:
class GoDataset(Dataset):
  def __init__(self, dataset):
    self.data = dataset

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    if 'pts_clicks' in self.data[idx]:
      return np.array(self.data[idx]['image']), np.array(self.data[idx]['pts_clicks'])
    else:
      return np.array(self.data[idx]['image']), np.zeros((4, 2))

In [ ]:
go_dataset = GoDataset(dataset)
train_size = int(0.8 * len(go_dataset))
val_size = len(go_dataset) - train_size
train_go, val_go = torch.utils.data.random_split(go_dataset, [train_size, val_size])

In [ ]:
!mkdir datasets
!mkdir datasets/data
!mkdir datasets/data/train
!mkdir datasets/data/val

mkdir: cannot create directory ‘datasets’: File exists
mkdir: cannot create directory ‘datasets/data’: File exists
mkdir: cannot create directory ‘datasets/data/train’: File exists
mkdir: cannot create directory ‘datasets/data/val’: File exists


In [ ]:
for i in range(len(train_go)):
  img, corners = train_go[i]

  if np.all(corners == 0):
    continue

  im = Image.fromarray(img)
  im.save(f"datasets/data/train/image{i}.jpg")

  y, x = img.shape[:2]

  np.savetxt(f"datasets/data/train/image{i}.txt", np.hstack([np.arange(4).reshape(4, 1),
                                                  corners / np.array([x, y]),
                                                  np.ones((4, 2)) * 0.05]))

In [ ]:
for i in range(len(val_go)):
  img, corners = val_go[i]

  if np.all(corners == 0):
    continue

  im = Image.fromarray(img)
  im.save(f"datasets/data/val/image{i}.jpg")

  y, x = img.shape[:2]

  corners / np.array([x, y])

  np.savetxt(f"datasets/data/val/image{i}.txt", np.hstack([np.arange(4).reshape(4, 1),
                                                  corners / np.array([x, y]),
                                                  np.ones((4, 2)) * 0.05]))

In [ ]:
# Writing the specified data.yaml file using Python

yaml_path = 'data.yaml'  # Define the file path (adjust as needed)

with open(yaml_path, 'w') as file:
    file.write(
        "# data.yaml\n"
        "train: data/train  # path to the folder containing training images and labels\n"
        "val: data/val \n"
        "nc: 4 \n"
        "names: ['top-left', 'top-right', 'bottom-right', 'bottom-left']\n"
    )

print(f"Created 'data.yaml' at: {yaml_path}")

Created 'data.yaml' at: data.yaml


In [ ]:
model = YOLO('yolov8n.pt')

results = model.train(
  data='data.yaml',
  imgsz=640,
  epochs=3,
  batch=2,
  name='yolov8n_corners')

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=3, time=None, patience=50, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_corners4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=tor

train: Scanning /content/datasets/data/train... 840 images, 0 backgrounds, 0 corrupt: 100%|██████████| 840/840 [00:00<00:00, 1758.25it/s]

train: New cache created: /content/datasets/data/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/data/val... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<00:00, 1101.21it/s]


...

                   all        210        840      0.872      0.867      0.936      0.523

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3     0.493G      1.545      1.295     0.9769         13        640: 100%|██████████| 420/420 [00:55<00:00,  7.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:07<00:00,  6.64it/s]


                   all        210        840      0.938      0.907      0.981      0.612

3 epochs completed in 0.056 hours.
Optimizer stripped from runs/detect/yolov8n_corners4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_corners4/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_corners4/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:06<00:00,  7.60it/s]


                   all        210        840      0.939      0.907      0.981      0.613
              top-left        210        210      0.984      0.877      0.984      0.654
             top-right        210        210      0.937      0.976      0.979      0.526
          bottom-right        210        210      0.882      0.959      0.987      0.614
           bottom-left        210        210      0.951      0.814      0.974      0.659
Speed: 0.5ms preprocess, 8.0ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/yolov8n_corners4


In [ ]:
import gc
gc.collect()

34

In [ ]:
def detect_corners_yolo(image, model):
    # YOLO model trained to detect corners on a chessboard
    model_trained = model.cpu()
    results = model_trained.predict(source=image, line_thickness=1, conf=0.25, save_txt=True, save=True)

    # get the corners coordinates from the model
    boxes = results[0].boxes
    arr = boxes.xywh.numpy()
    points = arr[:,0:2]
    corners = order_points(points)

    image_arr = cv2.imread(image)

    return corners, image_arr

In [ ]:
cell_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.Resize(size=(32, 32), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

state_converter = {'.': 0, 'W': 1, 'B': 2}
def warped_board_cell_splitter(warped, state, board_size=19):
    cells = []
    cells_state= []
    cell_size = warped.shape[0] / (4 * board_size), warped.shape[1] / (4 * board_size)
    y_grid = np.linspace(0, warped.shape[0], board_size)
    x_grid = np.linspace(0, warped.shape[1], board_size)
    for x in range(board_size):
        for y in range(board_size):
            y_min, y_max = np.clip([int(y_grid[y] - cell_size[0]), int(y_grid[y] + cell_size[0])], 0, warped.shape[0]).tolist()
            x_min, x_max = np.clip([int(x_grid[x] - cell_size[1]), int(x_grid[x] + cell_size[1])], 0, warped.shape[1]).tolist()
            cell = warped[y_min : y_max, x_min : x_max]
            cur_state = state_converter[state[y * (board_size * 2) + (x * 2)]]
            cells.append(cell)
            cells_state.append(cur_state)
    return cells, cells_state

In [ ]:
weights = ResNet18_Weights.DEFAULT
preprocess = weights.transforms()
res = torch.load('./model.pt').to(device)

In [ ]:
fs = []
path = './dataset2/'
for dirname, _, filenames in tqdm.notebook.tqdm(os.walk(path)):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.JPG') or filename.endswith('.png'):
            if not os.path.exists(os.path.join(dirname, 'board_extractor_state.yml')):
                continue
            gc.collect()
            if os.path.exists(os.path.join(dirname, filename[:-4] + '.txt')):
                annotation = read_annot(os.path.join(dirname, filename[:-4] + '.txt'))

            if os.path.exists(os.path.join(dirname, filename[:-4] + '.TXT')):
                annotation = read_annot(os.path.join(dirname, filename[:-4] + '.TXT'))

            full_name = os.path.join(dirname, filename)
            corners, image = detect_corners_yolo(full_name, model)
            warped = four_point_transform(image, corners)
            cells, cell_splits = warped_board_cell_splitter(warped, annotation)
            preds = []
            for cell in cells:
                post_cell = preprocess(cell_transforms(cv2.cvtColor(cell, cv2.COLOR_BGR2RGB))[None, :]).to(device)
                preds.append(int(res(post_cell).to('cpu').argmax(1)[0]))

            with open(full_name + '_result', "w") as f:
                print(preds, file=f)
                print(cell_splits, file=f)
            print(full_name)


0it [00:00, ?it/s]

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'line_width' instead.

image 1/1 /content/dataset2/Zingoni_Shakhov1/MLC_6708.JPG: 448x640 1 top-left, 1 bottom-right, 1 bottom-left, 165.9ms
Speed: 6.7ms preprocess, 165.9ms inference, 34.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/yolov8n_corners42
1 label saved to runs/detect/yolov8n_corners42/labels


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


./dataset2/Zingoni_Shakhov1/MLC_6708.JPG
WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'line_width' instead.

image 1/1 /content/dataset2/Zingoni_Shakhov1/MLC_6729.JPG: 448x640 1 top-left, 1 bottom-right, 1 bottom-left, 139.2ms
Speed: 3.9ms preprocess, 139.2ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/yolov8n_corners43
1 label saved to runs/detect/yolov8n_corners43/labels


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


./dataset2/Grazzini-Bevegni2/P1010730.JPG
WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'line_width' instead.

image 1/1 /content/dataset2/Grazzini-Bevegni2/P1010739.JPG: 480x640 1 top-left, 2 top-rights, 2 bottom-lefts, 94.9ms
Speed: 3.4ms preprocess, 94.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/yolov8n_corners4566
1 label saved to runs/detect/yolov8n_corners4566/labels


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


./dataset2/Grazzini-Bevegni2/P1010739.JPG
WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'line_width' instead.

image 1/1 /content/dataset2/19/Zrzut ekranu 2020-05-26 o 11.37.23.png: 448x640 (no detections), 89.6ms
Speed: 2.9ms preprocess, 89.6ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/yolov8n_corners4567
0 label saved to runs/detect/yolov8n_corners4567/labels


ValueError: ignored

In [ ]:
plt.imshow(image)
plt.scatter([pt[0] for pt in corners], [pt[1] for pt in corners], c='red', s=100)
plt.show()